<a href="https://colab.research.google.com/github/Troublem1/Portable_Water_Classification/blob/main/Water_Potability_Classification_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -r /content/drive/MyDrive/workSpace/Potable_Water/configs/PW_requirements.txt

     |████████████████████████████████| 74 kB 2.1 MB/s 
     |████████████████████████████████| 64.4 MB 10 kB/s 
     |████████████████████████████████| 61 kB 559 kB/s 
     |████████████████████████████████| 1.8 MB 74.4 MB/s 
     |████████████████████████████████| 5.8 MB 19.3 MB/s 
     |████████████████████████████████| 72 kB 1.1 MB/s 
     |████████████████████████████████| 22.3 MB 1.3 MB/s 
     |████████████████████████████████| 112 kB 73.5 MB/s 
     |████████████████████████████████| 636 kB 59.3 MB/s 
     |████████████████████████████████| 574 kB 59.1 MB/s 
     |████████████████████████████████| 9.5 MB 17.1 MB/s 


In [3]:
from omegaconf import OmegaConf
# from zindi.user import Zindian
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import shutil

import catboost as cb
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
# import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc,roc_auc_score, log_loss, f1_score
from sklearn import preprocessing
import seaborn as sns
import joblib
import matplotlib.pyplot as plt
from google.colab import files
import time
# import tensorflow as tf

In [4]:
os.makedirs('/content/drive/MyDrive/workSpace/Potable_Water/configs/' , exist_ok=True )

In [39]:
params = {'defaults':{'seed_value':2021, 'nKfold':10},
'dir': {'model_name':'003',
        'data_path':'/content/drive/MyDrive/workSpace/Potable_Water/data/',
        'train_file':'/content/drive/MyDrive/workSpace/Potable_Water/data/Train.csv' ,
        'test_file':'/content/drive/MyDrive/workSpace/Potable_Water/data/Test.csv' ,
        'submit_file':'/content/drive/MyDrive/workSpace/Potable_Water/data/SampleSubmission.csv',
        'model_path':'/content/drive/MyDrive/workSpace/Potable_Water/models/',
        'checkpoints_path':'/content/drive/MyDrive/workSpace/Potable_Water/checkpoints/',
        'predictions_path':'/content/drive/MyDrive/workSpace/Potable_Water/submissions/'},

'preprocessing':{'to_drop':['region_area_'],
               'feature_cols':['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity',
                               'Organic_carbon', 'Trihalomethanes', 'Turbidity'],
               'category_cols':['SW_range', 'alkalinity_range'],
               'target_col':'Potability'},  
   #sub1          
  'model1':{'learning_rate':0.08, 'n_estimators':96,'cat_smooth':10,'metrics':'binary_error',   'scale_pos_weight': 10.6 ,'max_depth':16, 'seed':2021,
                               'num_leaves':49,'reg_lambda':0.3},
   #pseudo label mdoel       
  'model2':{'learning_rate':0.08, 'n_estimators':96,'cat_smooth':10,'metrics':'binary_error', 'scale_pos_weight': 10.6 ,'max_depth':16, 'seed':2021,
                                'num_leaves':49,'reg_lambda':0.3, 'reg_alpha':0.3},
          
  #engineered category feaures model
  'model3': {'learning_rate':0.08, 'n_estimators':96,'cat_smooth':10,'metrics':'binary_error', 'scale_pos_weight': 10.6 ,'max_depth':16, 'seed':2021,
                               'num_leaves':22,'reg_lambda':0.3}
           }

if '003PW.yaml' not in os.listdir('/content/drive/MyDrive/workSpace/Potable_Water/configs/'):
  config_ = OmegaConf.load('/content/drive/MyDrive/workSpace/Potable_Water/configs/003PW.yaml')
else:
  OmegaConf.save(config=params, f='/content/drive/MyDrive/workSpace/Potable_Water/configs/003PW.yaml')
  config_ = OmegaConf.load('/content/drive/MyDrive/workSpace/Potable_Water/configs/003PW.yaml')


In [8]:
"try using these features in another competition"
#  neg_bagging_fraction=0.8,pos_bagging_fraction=0.7,

'try using these features in another competition'

In [9]:
print(OmegaConf.to_yaml(config_.dir))

model_name: '003'
data_path: /content/drive/MyDrive/workSpace/Potable_Water/data/
train_file: /content/drive/MyDrive/workSpace/Potable_Water/data/Train.csv
test_file: /content/drive/MyDrive/workSpace/Potable_Water/data/Test.csv
submit_file: /content/drive/MyDrive/workSpace/Potable_Water/data/SampleSubmission.csv
model_path: /content/drive/MyDrive/workSpace/Potable_Water/models/
checkpoints_path: /content/drive/MyDrive/workSpace/Potable_Water/checkpoints/
predictions_path: /content/drive/MyDrive/workSpace/Potable_Water/submissions/



In [10]:
os.makedirs(config_.dir.model_path , exist_ok=True )
os.makedirs(config_.dir.predictions_path, exist_ok=True)

In [11]:
# # create a user object
# my_username = "ZzyZx"
# user = Zindian(username = my_username)

# # Select a Zindi challenge
# user.select_a_challenge()  #current index 5
# user.which_challenge

# # Download the dataset of the selected challenge for the sfc select 2
# user.download_dataset(destination=config_.dir.data_path)

In [12]:
print(OmegaConf.to_yaml(config_.defaults))

seed_value: 2021
nKfold: 10



In [13]:
# Setting SEED to Reproduce Same Results even with "GPU"
import os
os.environ['PYTHONHASHSEED'] = str(config_.defaults.seed_value)
import random
random.seed(config_.defaults.seed_value)
import numpy as np
np.random.seed(config_.defaults.seed_value)


In [14]:
print(OmegaConf.to_yaml(config_.dir))

model_name: '003'
data_path: /content/drive/MyDrive/workSpace/Potable_Water/data/
train_file: /content/drive/MyDrive/workSpace/Potable_Water/data/Train.csv
test_file: /content/drive/MyDrive/workSpace/Potable_Water/data/Test.csv
submit_file: /content/drive/MyDrive/workSpace/Potable_Water/data/SampleSubmission.csv
model_path: /content/drive/MyDrive/workSpace/Potable_Water/models/
checkpoints_path: /content/drive/MyDrive/workSpace/Potable_Water/checkpoints/
predictions_path: /content/drive/MyDrive/workSpace/Potable_Water/submissions/



In [37]:
train_df = pd.read_csv(config_.dir.train_file)
test_df = pd.read_csv(config_.dir.test_file)
sub_df = pd.read_csv(config_.dir.submit_file)

In [16]:
train_df.head()

,region_area_,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,region_area_1,3.716080,180.196811,18630.362669,6.330435,NaN,602.990359,15.285013,56.444076,4.500656,0
1,region_area_100,4.270716,217.611599,24405.555346,7.086700,297.951852,359.623910,13.583526,54.589840,3.763906,0
2,region_area_1000,6.648849,232.462013,30733.808648,5.656462,344.670335,267.809179,17.863444,35.411181,4.827527,0
3,region_area_1001,NaN,261.753637,21174.713291,7.314074,NaN,582.848407,15.215849,90.367455,4.217553,0
4,region_area_1002,9.323584,263.484560,15751.480111,5.196259,297.593625,409.878704,10.727265,74.669285,3.006650,0


In [17]:
test_df.head()

,region_area_,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
0,region_area_0,NaN,255.664346,20791.623791,6.995401,369.261555,574.413654,10.484783,87.105970,2.963135
1,region_area_10,7.360640,216.294688,32452.919220,7.245890,327.369467,435.488419,15.691810,78.855016,3.662292
2,region_area_1006,7.230845,228.348694,17864.994196,5.513418,344.338446,377.794992,20.017950,59.418053,5.622018
3,region_area_1010,4.797460,241.089018,21280.551369,5.883692,327.539307,367.541193,11.818821,63.978068,4.038503
4,region_area_1011,9.319176,250.107456,14294.545161,7.480130,366.796865,348.546450,15.320293,63.506468,4.129230


In [18]:
sub_df.head()

,region_area_,Potability
0,region_area_0,1
1,region_area_10,1
2,region_area_1006,1
3,region_area_1010,1
4,region_area_1011,1


Variable Descriptions

- pH value: PH is an important parameter in evaluating the acid–base balance of water. It is also the indicator of acidic or alkaline condition of water status. WHO has recommended a maximum permissible limit of pH from 6.5 to 8.5. The current investigation ranges were 6.52–6.83 which are in the range of WHO standards.
- Hardness: Hardness is mainly caused by calcium and magnesium salts. These salts are dissolved from geologic deposits through which water travels. The length of time water is in contact with hardness producing material helps determine how much hardness there is in raw water. Hardness was originally defined as the capacity of water to precipitate soap caused by Calcium and Magnesium.
- Solids (Total dissolved solids - TDS): Water has the ability to dissolve a wide range of inorganic and some organic minerals or salts such as potassium, calcium, sodium, bicarbonates, chlorides, magnesium, sulfates etc. These minerals produced an unwanted taste and diluted color in the appearance of water. This is the important parameter for the use of water. The water with high TDS value indicates that water is highly mineralized. The desired limit for TDS is 500 mg/l and maximum limit is 1000 mg/l which is prescribed for drinking purposes.
- Chloramines: Chlorine and chloramine are the major disinfectants used in public water systems. Chloramines are most commonly formed when ammonia is added to chlorine to treat drinking water. Chlorine levels up to 4 milligrams per liter (mg/L or 4 parts per million (ppm)) are considered safe in drinking water.
- Sulfate: Sulfates are naturally occurring substances that are found in minerals, soil, and rocks. They are present in ambient air, groundwater, plants, and food. The principal commercial use of sulfate is in the chemical industry. Sulfate concentration in seawater is about 2,700 milligrams per liter (mg/L). It ranges from 3 to 30 mg/L in most freshwater supplies, although much higher concentrations (1000 mg/L) are found in some geographic locations.
- Conductivity: Pure water is not a good conductor of electric current rather it's a good insulator. Increase in ions concentration enhances the electrical conductivity of water. Generally, the amount of dissolved solids in water determines the electrical conductivity. Electrical conductivity (EC) actually measures the ionic process of a solution that enables it to transmit current. According to WHO standards, EC value should not exceed 400 μS/cm.
- Organic_carbon: Total Organic Carbon (TOC) in source waters comes from decaying natural organic matter (NOM) as well as synthetic sources. TOC is a measure of the total amount of carbon in organic compounds in pure water. According to the US EPA < 2 mg/L as TOC in treated / drinking water, and < 4 mg/Lit in source water which is used for treatment.
- Trihalomethanes: THMs are chemicals which may be found in water treated with chlorine. The concentration of THMs in drinking water varies according to the level of organic material in the water, the amount of chlorine required to treat the water, and the temperature of the water that is being treated. THM levels up to 80 ppm is considered safe in drinking water.
- Turbidity: The turbidity of water depends on the quantity of solid matter present in the suspended state. It is a measure of light emitting properties of water and the test is used to indicate the quality of waste discharge with respect to colloidal matter. The mean turbidity value obtained for Wondo Genet Campus (0.98 NTU) is lower than the WHO recommended value of 5.00 NTU.
- Potability: Indicates if water is safe for human consumption where 1 means Potable and 0 means Not potable.

In [19]:
def ph_scaler(df):
  "PH condition  6.52–6.83: non portable < portable < toxic"
  conditions5 = [
      (df['ph'] < 6.52),
      (df['ph'] >= 6.52) & (df['ph'] <= 6.83),
      (df['ph'] > 8.5)  
      ]
  values5 = ['-1','0','1']
  
  "ACIDIC < NEURAL < BASIC"
  conditions2 = [
      (df['ph'] < 7),
      (df['ph'] == 7),
      (df['ph'] > 7)  
      ]
  values2 = ['acidic','neutral','basic']

  df['SW_range'] = np.select(conditions5, values5)
  df['alkalinity_range'] = np.select(conditions2, values2)

def func(fn):
  "takes a func and performs the opration with the function"
  d_ = fn

  "returns preprocessed dataframe"
  return d_


001 SUBS

DATA AND ML APPROACHES ON DIFFERENT SUBMISSIONS

### 1st SUBMISSION CV:0.5808 LB 0.59106

In [40]:
#Here the cross validation by using the StratifiedKfold used to split the data into train,test and validation and repeatedly and Using the LGBMClassifier as Algorithm for building the model 
skf =StratifiedKFold(n_splits=10, shuffle=True,random_state=2021) # for cross validation by StratifiedKfold
scores = [] #list to store the sores obtained in training the model 
preds= [] #list to store the predictions 

X = train_df.drop(columns=['region_area_','Potability']) #config_.preprocess.to_drop+config_.preprocess.target)
y = train_df['Potability'] #[config_.preprocess.target[0]].astype(int)

test_new = test_df.drop(columns=['region_area_']) #config_.preprocess.to_drop)
test_fields = test_df['region_area_'] #[config_.preprocess.to_drop[0]]  #field

#creating a for loop for the stratified k fold to perform the cross validation 
i = 1
for train, test in skf.split(X, y):
    x_train, x_test= X.iloc[train], X.iloc[test] #spliting the data for x 
    y_train, y_test = y.iloc[train],y.iloc[test] #spliting the data for y 
    # model = lgb.LGBMClassifier(learning_rate=0.08, n_estimators = 96,cat_smooth=10,metrics='binary_error',
    #                           #  neg_bagging_fraction=0.8,pos_bagging_fraction=0.7,
    #                            scale_pos_weight= 10.6 ,max_depth=16, seed =2021,
    #                            num_leaves=49,reg_lambda=0.3) #Creating the model and tuning it with the different parameter 
    model = lgb.LGBMClassifier(**config_.model1)
    model.fit(x_train, y_train)# fitting  the model on train data
    print(i,'Split Trained') #print the splits that trained 
    score = f1_score(y_test,model.predict(x_test)) # checking the accuracy of the model 
    pred = model.predict_proba(test_new)[:,1] # making prediction
    scores.append(score)
    preds.append(pred)
    i += 1
print('Total Mean Score Obtained :',np.mean(scores))

preds_mean = np.mean(preds, axis=0) # mean of all the predictions
final = [] #list to store the final results 
for x in preds_mean:
    if x >= 0.51:
        final.append(1)
    else:
        final.append(0)

submit = test_df.copy() #Copy the submission file to the variable submit 
submit['Potability'] = final
submit[['region_area_','Potability']].to_csv('003_1.csv',index=False) #creating and naming the file ready for submission

1 Split Trained
2 Split Trained
3 Split Trained
4 Split Trained
5 Split Trained
6 Split Trained
7 Split Trained
8 Split Trained
9 Split Trained
10 Split Trained
Total Mean Score Obtained : 0.5808120490570688


### 2nd SUBMISSION PSUEDO LABELING CV:0.703 LB: 0.588

In [41]:
#Here the cross validation by using the StratifiedKfold used to split the data into train,test and validation and repeatedly and Using the LGBMClassifier as Algorithm for building the model 
skf =StratifiedKFold(n_splits=10, shuffle=True,random_state=2021) # for cross validation by StratifiedKfold
scores = [] #list to store the sores obtained in training the model 
preds= [] #list to store the predictions 

X = train_df.drop(columns=['region_area_','Potability']) #config_.preprocess.to_drop+config_.preprocess.target)
y = train_df['Potability'] #[config_.preprocess.target[0]].astype(int)

test_new = test_df.drop(columns=['region_area_']) #config_.preprocess.to_drop)
test_fields = test_df['region_area_'] #[config_.preprocess.to_drop[0]]  #field

#creating a for loop for the stratified k fold to perform the cross validation 
i = 1
for train, test in skf.split(X, y):
    x_train, x_test= X.iloc[train], X.iloc[test] #spliting the data for x 
    y_train, y_test = y.iloc[train],y.iloc[test] #spliting the data for y 
    # model = lgb.LGBMClassifier(learning_rate=0.08, n_estimators = 96,cat_smooth=10,metrics='binary_error',
    #                           #  neg_bagging_fraction=0.8,pos_bagging_fraction=0.7,
    #                            scale_pos_weight= 10.6 ,max_depth=16, seed =2021,
    #                            num_leaves=49,reg_lambda=0.3) #Creating the model and tuning it with the different parameter 
    model = lgb.LGBMClassifier(**config_.model1)
    model.fit(x_train, y_train)# fitting  the model on train data
    print(i,'Split Trained') #print the splits that trained 
    score = f1_score(y_test,model.predict(x_test)) # checking the accuracy of the model 
    pred = model.predict_proba(test_new)[:,1] # making prediction
    scores.append(score)
    preds.append(pred)
    i += 1
print('Total Mean Score Obtained :',np.mean(scores))

preds_mean = np.mean(preds, axis=0) # mean of all the predictions
final = [] #list to store the final results 
for x in preds_mean:
    if x >= 0.51:
        final.append(1)
    else:
        final.append(0)

# submit = test_df.copy() #Copy the submission file to the variable submit 
# submit['Potability'] = final
# submit[['region_area_','Potability']].to_csv('003xviibaseline4.csv',index=False) #creating and naming the file ready for submission

1 Split Trained
2 Split Trained
3 Split Trained
4 Split Trained
5 Split Trained
6 Split Trained
7 Split Trained
8 Split Trained
9 Split Trained
10 Split Trained
Total Mean Score Obtained : 0.5808120490570688


In [43]:
test_df['Potability'] = final

new_train = pd.concat([train_df,test_df]).reset_index(drop=True)

In [44]:
#Here the cross validation by using the StratifiedKfold used to split the data into train,test and validation and repeatedly and Using the LGBMClassifier as Algorithm for building the model 
skf =StratifiedKFold(n_splits=10, shuffle=True,random_state=2021) # for cross validation by StratifiedKfold
scores = [] #list to store the sores obtained in training the model 
preds= [] #list to store the predictions 

X = new_train.drop(columns=['region_area_','Potability']) #config_.preprocess.to_drop+config_.preprocess.target)
y = new_train['Potability'] #[config_.preprocess.target[0]].astype(int)

test_new = test_df.drop(columns=['region_area_', 'Potability']) #config_.preprocess.to_drop)
test_fields = test_df['region_area_'] #[config_.preprocess.to_drop[0]]  #field

#creating a for loop for the stratified k fold to perform the cross validation 
i = 1
for train, test in skf.split(X, y):
    x_train, x_test= X.iloc[train], X.iloc[test] #spliting the data for x 
    y_train, y_test = y.iloc[train],y.iloc[test] #spliting the data for y 
    # model = lgb.LGBMClassifier(learning_rate=0.08, n_estimators = 96,cat_smooth=10,metrics='binary_error',
    #                           #  neg_bagging_fraction=0.8,pos_bagging_fraction=0.7,
    #                            scale_pos_weight= 10.6 ,max_depth=16, seed =2021,
    #                            num_leaves=49,reg_lambda=0.3, reg_alpha=0.3) #Creating the model and tuning it with the different parameter 
    model = lgb.LGBMClassifier(**config_.model2)
    model.fit(x_train, y_train)# fitting  the model on train data
    print(i,'Split Trained') #print the splits that trained 
    score = f1_score(y_test,model.predict(x_test)) # checking the accuracy of the model 
    pred = model.predict_proba(test_new)[:,1] # making prediction
    scores.append(score)
    preds.append(pred)
    i += 1
print('Total Mean Score Obtained :',np.mean(scores))

preds_mean = np.mean(preds, axis=0) # mean of all the predictions
final = [] #list to store the final results 
for x in preds_mean:
    if x >= 0.51:
        final.append(1)
    else:
        final.append(0)

submit = test_df.copy() #Copy the submission file to the variable submit 
submit['Potability'] = final
submit[['region_area_','Potability']].to_csv('003_2.csv',index=False) #creating and naming the file ready for submission

1 Split Trained
2 Split Trained
3 Split Trained
4 Split Trained
5 Split Trained
6 Split Trained
7 Split Trained
8 Split Trained
9 Split Trained
10 Split Trained
Total Mean Score Obtained : 0.7032938196522149


In [45]:
test_df.drop(columns='Potability' ,inplace=True)

### SUB 3 FEATURE ENGINEEERED FEATURES MODEL cv 0.578 LB: 0.5889

In [46]:
func(ph_scaler(train_df))   #ph category cols
func(ph_scaler(test_df))   #ph category cols

train_df[['SW_range', 'alkalinity_range']] = train_df[['SW_range', 'alkalinity_range']].astype('category')
test_df[['SW_range', 'alkalinity_range']] = test_df[['SW_range', 'alkalinity_range']].astype('category')

In [47]:
#Here the cross validation by using the StratifiedKfold used to split the data into train,test and validation and repeatedly and Using the LGBMClassifier as Algorithm for building the model 
skf =StratifiedKFold(n_splits=10, shuffle=True,random_state=2021) # for cross validation by StratifiedKfold
scores = [] #list to store the sores obtained in training the model 
preds= [] #list to store the predictions 

X = train_df.drop(columns=['region_area_','Potability']) #config_.preprocess.to_drop+config_.preprocess.target)
y = train_df['Potability'] #[config_.preprocess.target[0]].astype(int)

test_new = test_df.drop(columns=['region_area_']) #config_.preprocess.to_drop)
test_fields = test_df['region_area_'] #[config_.preprocess.to_drop[0]]  #field

#creating a for loop for the stratified k fold to perform the cross validation 
i = 1
for train, test in skf.split(X, y):
    x_train, x_test= X.iloc[train], X.iloc[test] #spliting the data for x 
    y_train, y_test = y.iloc[train],y.iloc[test] #spliting the data for y 
    # model = lgb.LGBMClassifier(learning_rate=0.08, n_estimators = 96,cat_smooth=10,metrics='binary_error',
    #                            scale_pos_weight= 10.6 ,max_depth=16, seed =2021,
    #                            num_leaves=22,reg_lambda=0.3,) #Creating the model and tuning it with the different parameter 
    
    model = lgb.LGBMClassifier(**config_.model3)
    model.fit(x_train, y_train)# fitting  the model on train data
    print(i,'Split Trained') #print the splits that trained 
    score = f1_score(y_test,model.predict(x_test)) # checking the accuracy of the model 
    pred = model.predict_proba(test_new)[:,1] # making prediction
    scores.append(score)
    preds.append(pred)
    i += 1
print('Total Mean Score Obtained :',np.mean(scores))

preds_mean = np.mean(preds, axis=0) # mean of all the predictions
final = [] #list to store the final results 
for x in preds_mean:
    if x >= 0.51:
        final.append(1)
    else:
        final.append(0)

submit = test_df.copy() #Copy the submission file to the variable submit 
submit['Potability'] = final
submit[['region_area_','Potability']].to_csv('003_3.csv',index=False) #creating and naming the file ready for submission

1 Split Trained
2 Split Trained
3 Split Trained
4 Split Trained
5 Split Trained
6 Split Trained
7 Split Trained
8 Split Trained
9 Split Trained
10 Split Trained
Total Mean Score Obtained : 0.5786560519655769
